In [29]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import sys
from importlib import reload
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import probdownscale
reload(probdownscale.TaskExtractor)
reload(probdownscale.MetaTrain)
from probdownscale.MetaTrain import MetaSGD

from probdownscale.TaskExtractor import TaskExtractor
import math
import numpy as np
import netCDF4 as nc

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow_probability import distributions as tfd

## 1. Debug TaskExtractor

In [30]:
file_path_g_05 = r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20050516-20060515.nc'
file_path_g_06 =  r'C:\Users\96349\Documents\Downscale_data\MERRA2\G5NR_aerosol_variables_over_MiddleEast_daily_20060516-20070515.nc'
file_path_m = r'C:\Users\96349\Documents\Downscale_data\MERRA2\MERRA2_aerosol_variables_over_MiddleEast_daily_20000516-20180515.nc'
target_var = 'BCSMASS'

# read data
g05_data = nc.Dataset(file_path_g_05)
g06_data = nc.Dataset(file_path_g_06)
m_data_nc = nc.Dataset(file_path_m)

# define lat&lon of MERRA, G5NR and mete
M_lons = m_data_nc.variables['lon'][:]
# self.M_lons = (M_lons-M_lons.mean())/M_lons.std()
M_lats = m_data_nc.variables['lat'][:]
# self.M_lats = (M_lats-M_lats.mean())/M_lats.std()
G_lons = g05_data.variables['lon'][:]
# self.G_lons = (G_lons-G_lons.mean())/G_lons.std()
G_lats = g05_data.variables['lat'][:]

# extract target data
g_data = np.concatenate((g05_data.variables[target_var][:], g06_data.variables[target_var][:]), axis=0)
m_data = m_data_nc.variables[target_var][5*365:7*365]

C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
C:\Users\96349\anaconda3\envs\Downscale_env\lib\site-packag

In [31]:
data = [g_data, m_data]
lats_lons = [G_lats, G_lons, M_lats, M_lons]
task_dim = 3
test_proportion = 0.3
n_lag = 10
taskextractor = TaskExtractor(data, lats_lons, task_dim, test_proportion, n_lag)

In [32]:
train_x, train_y, test_x, test_y, locations = taskextractor.get_random_tasks(10)


In [33]:
train_x, train_y, test_x, test_y, location = taskextractor._get_random_task(is_random=False, record=False, lat_lon=(13.5625, 49.375))

In [34]:
locations = taskextractor.get_grid_locations()

In [35]:
train_x, train_y, test_x, test_y, locations = taskextractor.get_random_tasks(locations=locations[:4])

In [36]:
len(train_x)

4

In [41]:
locations[5:10]

[]

In [40]:
10//3

3

## 2. Test Network

In [145]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow_probability import distributions as tfd

In [34]:
# the last channel is the comonents
alpha = np.random.rand(3,2,5)
alpha = alpha/alpha.sum()
mu = np.random.rand(3, 2, 5)
test_md = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha),
        components_distribution=tfd.Exponential(
        rate=mu))

In [43]:
log_like = test_md.log_prob(tf.reduce_mean(mu, axis=-1))
-tf.reduce_mean(log_like, axis=-1)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.95185266, 1.00110027, 1.33222622])>

In [44]:
tf.reduce_mean(mu, axis=-1)

<tf.Tensor: shape=(3, 2), dtype=float64, numpy=
array([[0.38051166, 0.57145636],
       [0.62442875, 0.36601753],
       [0.28161602, 0.40978898]])>

In [160]:
a =  np.random.rand(3,3,3)
#a = (a - a.min())/(a.max() - a.min())
a = a/a.sum()
a.sum()

1.0

In [17]:
def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

components = 3
no_parameters = 2
def slice_parameter_vectors(parameter_vector):
    return [parameter_vector[:, i * task_dim*task_dim*components:(i + 1) *task_dim*task_dim*components] for i in range(no_parameters)]

def gnll_loss(y, parameter_vector):
    alpha, mu = slice_parameter_vectors(parameter_vector)  # Unpack parameter vectors
    #print(alpha.shape, mu.shape)
    alpha1 = tf.reshape(alpha, (tf.shape(alpha)[0], task_dim, task_dim, components))
    mu1 = tf.reshape(mu, (tf.shape(mu)[0], task_dim, task_dim, components))
    #print(alpha1.shape, mu1.shape)
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha1),
        components_distribution=tfd.Exponential(
        rate=mu1)
    )

    log_likelihood = gm.log_prob(y)  # Evaluate log-probability of y
    
    return -tf.reduce_mean(log_likelihood, axis=-1)

tf.keras.utils.get_custom_objects().update({'nnelu': layers.Activation(nnelu)})

In [18]:
# MDN model
# input dim (time, channel, rows, cols)
input1 = layers.Input(shape=(n_lag, 1, task_dim, task_dim)) 
input2 = layers.Input(shape=(task_dim, task_dim, 1))
input3 = layers.Input(shape=(1))

X = layers.ConvLSTM2D(filters=20, kernel_size=(1,2), activation='tanh', return_sequences=False)(input1)
X = layers.Flatten()(X)

X1 = layers.Conv2D(20, (2,2), activation='tanh')(input2)
X1 = layers.Flatten()(X1)
X2 = layers.BatchNormalization()(input3)
X2 = layers.Dense(30, activation='relu')(X2)

X = layers.Concatenate()([X, X1, X2])

alphas = layers.Dense(components*task_dim*task_dim, activation="softmax")(X)
#alphas = layers.Reshape((task_dim, task_dim, components), name="alphas")(alphas)
mus = layers.Dense(components*task_dim*task_dim, activation="nnelu")(X)
#mus = layers.Reshape((task_dim, task_dim, components) ,name="mus")(mus)
#sigmas = layers.Dense(components, activation="nnelu", name="sigmas")(X)
output = layers.Concatenate()([alphas, mus])
MDN_model = Model([input1, input2, input3], output)
MDN_model.compile(optimizer='adam', loss=gnll_loss)
MDN_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 10, 1, 10,   0           []                               
                                10)]                                                              
                                                                                                  
 input_5 (InputLayer)           [(None, 10, 10, 1)]  0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 conv_lstm2d_1 (ConvLSTM2D)     (None, 1, 9, 20)     4880        ['input_4[0][0]']          

In [22]:
MDN_model.fit(train_x, train_y, epochs=100)

Epoch 1/100
16/16 [==============================] - 7s 244ms/step - loss: -0.0445
Epoch 2/100
16/16 [==============================] - 4s 257ms/step - loss: -0.4285
Epoch 3/100
16/16 [==============================] - 4s 263ms/step - loss: -1.3516
Epoch 4/100
16/16 [==============================] - 4s 264ms/step - loss: -2.3884
Epoch 5/100
16/16 [==============================] - 4s 264ms/step - loss: -3.0921
Epoch 6/100
16/16 [==============================] - 4s 260ms/step - loss: -3.4926
Epoch 7/100
16/16 [==============================] - 4s 264ms/step - loss: -3.7561
Epoch 8/100
16/16 [==============================] - 4s 262ms/step - loss: -3.9519
Epoch 9/100
16/16 [==============================] - 4s 262ms/step - loss: -4.1115
Epoch 10/100
16/16 [==============================] - 4s 263ms/step - loss: -4.2492
Epoch 11/100
16/16 [==============================] - 4s 280ms/step - loss: -4.3721
Epoch 12/100
16/16 [==============================] - 4s 265ms/step - loss: -4.4835
E

KeyboardInterrupt: 

## 3. Debug Meta Learning Algorithm

In [42]:
# define necessary tool functions
components = 5
no_parameters = 3
data = [g_data, m_data]
lats_lons = [G_lats, G_lons, M_lats, M_lons]
task_dim = 3
test_proportion = 0.5
n_lag = 10

def nnelu(input):
    return tf.add(tf.constant(1, dtype=tf.float32), tf.nn.elu(input))

def slice_parameter_vectors(parameter_vector):
    return [parameter_vector[:, i * task_dim*task_dim*components:(i + 1) *task_dim*task_dim*components] for i in range(no_parameters)]

def gnll_loss(y, parameter_vector):
    alpha, mu, sigma = slice_parameter_vectors(parameter_vector)  # Unpack parameter vectors
    #print(alpha.shape, mu.shape)
    alpha1 = tf.reshape(alpha, (tf.shape(alpha)[0], task_dim, task_dim, components))
    mu1 = tf.reshape(mu, (tf.shape(mu)[0], task_dim, task_dim, components))
    sigma1 = tf.reshape(sigma,  (tf.shape(sigma)[0], task_dim, task_dim, components))
    #print(alpha1.shape, mu1.shape)
    gm = tfd.MixtureSameFamily(
        mixture_distribution=tfd.Categorical(probs=alpha1),
        components_distribution=tfd.Gamma(
        concentration=mu1, rate=sigma1)
    )

    log_likelihood = gm.log_prob(y)  # Evaluate log-probability of y
    #print(log_likelihood)
    return -tf.reduce_mean(log_likelihood, axis=-1)

tf.keras.utils.get_custom_objects().update({'nnelu': layers.Activation(nnelu)})

# define MDN model
# input dim (time, channel, rows, cols)
input1 = layers.Input(shape=(n_lag, 1, task_dim, task_dim)) 
input2 = layers.Input(shape=(task_dim, task_dim, 1))
input3 = layers.Input(shape=(1))

X = layers.ConvLSTM2D(filters=20, kernel_size=(1,2), activation='tanh', return_sequences=True)(input1)
#X = layers.ConvLSTM2D(filters=20, kernel_size=(1,2), activation='relu', return_sequences=True)(X)
#X = layers.ConvLSTM2D(filters=20, kernel_size=(1,1), activation='relu')(X)
X = layers.Flatten()(X)
X = layers.Dense(512, activation='relu')(X)
X = layers.BatchNormalization()(X)
X = layers.Dense(300, activation='relu')(X)

X1 = layers.Conv2D(20, (2,2), activation='tanh')(input2)
X1 = layers.Flatten()(X1)
X2 = layers.BatchNormalization()(input3)
X2 = layers.Dense(30, activation='relu')(X2)

X = layers.Concatenate()([X, X1, X2])

alphas = layers.Dense(components*task_dim*task_dim, activation="softmax")(X)
#alphas = layers.Reshape((task_dim, task_dim, components), name="alphas")(alphas)
mus = layers.Dense(components*task_dim*task_dim, activation='nnelu')(X)
#mus = layers.Reshape((task_dim, task_dim, components) ,name="mus")(mus)
sigmas = layers.Dense(components*task_dim*task_dim, activation="nnelu", name="sigmas")(X)
output = layers.Concatenate()([alphas, mus, sigmas])
MDN_model = Model([input1, input2, input3], output)

# define TaskExtractor

taskextractor = TaskExtractor(data, lats_lons, task_dim, test_proportion, n_lag)

In [50]:
reload(probdownscale.MetaTrain)
from probdownscale.MetaTrain import MetaSGD
# define meta learner
meta_step = 10
meta_optimizer = tf.keras.optimizers.RMSprop(0.1)
inner_step = 2
inner_optimizer = tf.keras.optimizers.SGD()

meta_learner = MetaSGD(MDN_model, gnll_loss, meta_step, meta_optimizer, inner_step, inner_optimizer, taskextractor)

In [161]:
def inner_rate_function(inner_rate, batch_size, inner_step):
    return inner_rate/inner_step*math.log(batch_size, 20)

def meta_rate_function(meta_rate, batch_locations, seen_locations, covariance_function, distance_function):
    batch_size = len(batch_location)
    center = np.average(list(seen_locations.keys()), weights=list(seen_locations.values()), axis=0)
    batch_dist = np.mean([distance_function(locat, center) for locat in batch_locations])
    
inner_rate_f(0.01, 100, 5)

0.003074487147360964

In [51]:
meta_learner.meta_fit(5, basic_train=False, bootstrap_train=True)

Now doing bootstrap training step: 0 / 8646 loss:  -18.561356
Now doing bootstrap training step: 1 / 8646 loss:  -17.011003
Now doing bootstrap training step: 2 / 8646 loss:  -16.69801
Now doing bootstrap training step: 3 / 8646 loss:  -13.284741
Now doing bootstrap training step: 4 / 8646 loss:  -18.099634
Now doing bootstrap training step: 5 / 8646 loss:  -18.658218
Now doing bootstrap training step: 6 / 8646 loss:  -18.285648
Now doing bootstrap training step: 7 / 8646 loss:  -18.578285
Now doing bootstrap training step: 8 / 8646 loss:  -18.449034
Now doing bootstrap training step: 9 / 8646 loss:  -18.393436


[-18.561356,
 -17.011003,
 -16.69801,
 -13.284741,
 -18.099634,
 -18.658218,
 -18.285648,
 -18.578285,
 -18.449034,
 -18.393436]

In [185]:
np.mean([1,2,3])

2.0

In [149]:
while 1 > 0:
    print(meta_learner._train_on_batch(3, use_test_for_meta=True))

origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.1>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
(<tf.Tensor: shape=(), dtype=float32, numpy=-1.2884859>, [<tf.Tensor: shape=(), dtype=float32, numpy=-1.3128259>, <tf.Tensor: shape=(), dtype=float32, numpy=-1.3372735>, <tf.Tensor: shape=(), dtype=float32, numpy=-1.2153584>])
origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
(<tf.Tensor: shape=(), dtype=float32, numpy=-18.776932>, [<tf.Tensor: shape=(), dtype=float32, numpy=-18.566221>, <tf.Tensor: shape=(), dtype=float32, numpy=-19.05812>, <tf.Tensor: shape=(), dtype=float32, numpy=-18.706453>])
origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
(<tf.Tensor: shape=(), dtype=float32, numpy=-17.04699>, [<tf.Tensor:

origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
(<tf.Tensor: shape=(), dtype=float32, numpy=-18.691551>, [<tf.Tensor: shape=(), dtype=float32, numpy=-18.695385>, <tf.Tensor: shape=(), dtype=float32, numpy=-18.403854>, <tf.Tensor: shape=(), dtype=float32, numpy=-18.975414>])
origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
(<tf.Tensor: shape=(), dtype=float32, numpy=-18.489832>, [<tf.Tensor: shape=(), dtype=float32, numpy=-19.902456>, <tf.Tensor: shape=(), dtype=float32, numpy=-17.601349>, <tf.Tensor: shape=(), dtype=float32, numpy=-17.96569>])
origional lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
updated lr: <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.01>
(<tf.Tensor: shape=(), dtype=float32, numpy=-18.913248>, [<tf.Tenso

KeyboardInterrupt: 

In [429]:
for t in MDN_model.get_weights():
    print(t.shape)
    print(type(t))

(1, 2, 10, 80)
<class 'numpy.ndarray'>
(1, 2, 20, 80)
<class 'numpy.ndarray'>
(80,)
<class 'numpy.ndarray'>
(2, 2, 1, 20)
<class 'numpy.ndarray'>
(20,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1,)
<class 'numpy.ndarray'>
(1, 30)
<class 'numpy.ndarray'>
(30,)
<class 'numpy.ndarray'>
(1830, 10000)
<class 'numpy.ndarray'>
(10000,)
<class 'numpy.ndarray'>
(1830, 10000)
<class 'numpy.ndarray'>
(10000,)
<class 'numpy.ndarray'>


In [378]:
for t in MDN_model.trainable_variables:
    print(t.shape)

(1, 2, 10, 80)
(1, 2, 20, 80)
(80,)
(2, 2, 1, 20)
(20,)
(1,)
(1,)
(1, 30)
(30,)
(1830, 10000)
(10000,)
(1830, 10000)
(10000,)


In [411]:
np.random.rand()

0.7485470997143776